# WORKFLOW 2

In [1]:
# IMPORTS
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json

In [2]:
# GLOBAL VARIABLES
execution_count = 1  # Adjust as needed
window_size = 100
window_size_std = 400

dates = ["2025-02-03", "2025-02-04", "2025-02-05", "2025-02-06", "2025-02-07", "2025-02-08", "2025-02-09"]

folder_to_save = "./experiment_plots"

optimized_color = sns.color_palette("Blues", n_colors=1)[0]  # Main blue shade
baseline_color = sns.color_palette("Oranges", n_colors=1)[0]  # Main orange shade


theme_style = "darkgrid"
color_palette_style = "deep"

In [3]:
# PREPARE DATA

# prepare optimized data
df_list = []
for date in dates:
    file_path = f'./logs_analysis/{date}/execution_{execution_count}/{date}-optimizedFunction-logs.csv'
    
    if os.path.exists(file_path):  # Check if file exists to avoid errors
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert timestamp to datetime
        df['day'] = df['timestamp'].dt.date  # Extract date to a new column
        df['duration_minutes'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds() / 60  # Normalize time
        df['smoothed_execution_time'] = df['execution_time'].rolling(window=window_size).mean()

        df_list.append(df)
# Concatenate all dataframes
df_all_days_optimized = pd.concat(df_list, ignore_index=True)


# prepare baseline data
df_list = []
for date in dates:
    file_path = f'./logs_analysis/{date}/execution_{execution_count}/{date}-baselineFunction-logs.csv'
    
    if os.path.exists(file_path):  # Check if file exists to avoid errors
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert timestamp to datetime
        df['day'] = df['timestamp'].dt.date  # Extract date to a new column
        df['duration_minutes'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds() / 60  # Normalize time
        df['smoothed_execution_time'] = df['execution_time'].rolling(window=window_size).mean()

        df_list.append(df)
# Concatenate all dataframes
df_all_days_baseline = pd.concat(df_list, ignore_index=True)



# GROUP ALL DAYS TOGETHER
df_optimized_all_days = df_all_days_optimized
# Sort the dataframe by duration_minutes to ensure proper rolling calculations
df_optimized_all_days = df_optimized_all_days.sort_values(by='duration_minutes')

df_baseline_all_days = df_all_days_baseline
# Sort the dataframe by duration_minutes to ensure proper rolling calculations
df_baseline_all_days = df_baseline_all_days.sort_values(by='duration_minutes')


# Compute rolling median & std for optimized function
df_optimized_all_days['smoothed_median'] = df_optimized_all_days['execution_time'].rolling(window=window_size, center=True).median()
df_optimized_all_days['smoothed_std'] = df_optimized_all_days['execution_time'].rolling(window=window_size_std, center=True).std()

# Compute rolling median & std for baseline function
df_baseline_all_days['smoothed_median'] = df_baseline_all_days['execution_time'].rolling(window=window_size, center=True).median()
df_baseline_all_days['smoothed_std'] = df_baseline_all_days['execution_time'].rolling(window=window_size_std, center=True).std()

# Drop NaNs
df_optimized_all_days = df_optimized_all_days.dropna(subset=['smoothed_median', 'smoothed_std'])
df_baseline_all_days = df_baseline_all_days.dropna(subset=['smoothed_median', 'smoothed_std'])

##### SINGULAR EXECUTIONS

In [ ]:
# OPTIMIZED EXECUTIONS
sns.set_theme(style=theme_style)
num_days = df_all_days_optimized["day"].nunique()
palette = sns.color_palette("light:b", n_colors=num_days+2)[1:-1]

# Create the plot
plt.figure(figsize=(12, 6))

# Plot execution time for different days
sns.lineplot(
    data=df_all_days_optimized, 
    x='duration_minutes', 
    y='smoothed_execution_time', 
    hue='day', 
    lw=2, 
    palette=palette
)

# Add labels and title
plt.xlabel('Duration (min)')
plt.ylabel('Optimized Execution Time (ms)')
plt.title('Optimized Execution Time Over Multiple Days (Smoothed)')

# Set x-axis and y-axis to start at 0
plt.xlim(left=0)
plt.ylim(bottom=0)

# Add a legend
plt.legend(title="Day")

# Shrink layout
plt.tight_layout()

# SAVE PLOT
plot_filename = os.path.join(folder_to_save, "execution-times-optimized.pdf")
plt.savefig(plot_filename)
# SHOW PLOT
plt.show()

In [ ]:
# BASELINE EXECUTIONS
sns.set_theme(style=theme_style)
# palette = sns.color_palette("YlOrBr", n_colors=df_all_days_optimized["day"].nunique())
num_days = df_all_days_optimized["day"].nunique()
palette = sns.color_palette("light:r", n_colors=num_days+2)[1:-1]

# Create the plot
plt.figure(figsize=(12, 6))

# Plot execution time for different days
sns.lineplot(
    data=df_all_days_optimized, 
    x='duration_minutes', 
    y='smoothed_execution_time', 
    hue='day', 
    lw=2, 
    palette=palette
)

# Add labels and title
plt.xlabel('Duration (min)')
plt.ylabel('Baseline Execution Time (ms)')
plt.title('Baseline Execution Time Over Multiple Days (Smoothed)')

# Set x-axis and y-axis to start at 0
plt.xlim(left=0)
plt.ylim(bottom=0)

# Add a legend
plt.legend(title="Day")

# Shrink layout
plt.tight_layout()

# SAVE PLOT
plot_filename = os.path.join(folder_to_save, "execution-times-baseline.pdf")
plt.savefig(plot_filename)
# SHOW PLOT
plt.show()

In [ ]:
# BASELINE AND OPTIMIZED EXECUTIONS IN ONE GRAPH
sns.set_theme(style=theme_style)
num_days = df_all_days_optimized["day"].nunique()

# Define color palettes
optimized_palette = sns.color_palette("light:b", n_colors=num_days+2)[1:-1]
baseline_palette = sns.color_palette("light:r", n_colors=num_days+2)[1:-1]

plt.figure(figsize=(12, 6))

# Plot optimized execution times
optimized_lines = sns.lineplot(
    data=df_all_days_optimized,
    x='duration_minutes',
    y='smoothed_execution_time',
    hue='day',
    lw=2,
    palette=optimized_palette
)

# Plot baseline execution times
baseline_lines = sns.lineplot(
    data=df_all_days_baseline,
    x='duration_minutes',
    y='smoothed_execution_time',
    hue='day',
    lw=2,
    palette=baseline_palette
)

# Labels and title
plt.xlabel('Duration (min)')
plt.ylabel('Execution Time (ms)')
plt.title('Optimized vs. Baseline Execution Time Over Multiple Days (Smoothed)')

# Set x-axis and y-axis to start at 0
plt.xlim(left=0)
plt.ylim(bottom=0)

# Add a legend
plt.legend(title="Day")

# Shrink layout
plt.tight_layout()

# SAVE PLOT
plot_filename = os.path.join(folder_to_save, "execution-times-optimized-and-baseline.pdf")
plt.savefig(plot_filename)

# SHOW PLOT
plt.show()

##### GROUPED

In [ ]:
# OPTIMIZED RESULTS GROUPED WITH ERROR BAND

# Plot the data
sns.set_theme(style=theme_style)
palette = sns.color_palette(color_palette_style)

plt.figure(figsize=(12, 6))

# Plot smoothed median execution time
sns.lineplot(
    data=df_optimized_all_days,
    x='duration_minutes',
    y='smoothed_median',
    color=palette[0],
    label='Optimized Function (Smoothed Median)',
    lw=2
)

# Add error band using standard deviation
plt.fill_between(
    df_optimized_all_days['duration_minutes'],
    df_optimized_all_days['smoothed_median'] - df_optimized_all_days['smoothed_std'],
    df_optimized_all_days['smoothed_median'] + df_optimized_all_days['smoothed_std'],
    color=palette[0],
    alpha=0.2,
    label='Standard Deviation'
)

# Add labels and title
plt.xlabel('Duration (min)')
plt.ylabel('Execution Time (ms)')
plt.title('Smoothed Median Execution Time with Error Band (Optimized Function)')

# Set x-axis and y-axis to start at 0
plt.xlim(left=0)
plt.ylim(bottom=0)

# Add a legend
plt.legend()

# Shrink layout
plt.tight_layout()

# SAVE PLOT
plot_filename = os.path.join(folder_to_save, "grouped-optimized-error-bands.pdf")
plt.savefig(plot_filename)

# Show plot
plt.show()

In [ ]:
# BASELINE RESULTS GROUPED WITH ERROR BAND

# Plot the data
sns.set_theme(style=theme_style)
palette = sns.color_palette(color_palette_style)

plt.figure(figsize=(12, 6))

# Plot smoothed median execution time
sns.lineplot(
    data=df_baseline_all_days,
    x='duration_minutes',
    y='smoothed_median',
    color=palette[1],
    label='Optimized Function (Smoothed Median)',
    lw=2
)

# Add error band using standard deviation
plt.fill_between(
    df_baseline_all_days['duration_minutes'],
    df_baseline_all_days['smoothed_median'] - df_baseline_all_days['smoothed_std'],
    df_baseline_all_days['smoothed_median'] + df_baseline_all_days['smoothed_std'],
    color=palette[1],
    alpha=0.2,
    label='Standard Deviation'
)

# Add labels and title
plt.xlabel('Duration (min)')
plt.ylabel('Execution Time (ms)')
plt.title('Smoothed Median Execution Time with Error Band (Optimized Function)')

# Set x-axis and y-axis to start at 0
plt.xlim(left=0)
plt.ylim(bottom=0)

# Add a legend
plt.legend()

# Shrink layout
plt.tight_layout()

# SAVE PLOT
plot_filename = os.path.join(folder_to_save, "grouped-baseline-error-bands.pdf")
plt.savefig(plot_filename)

# Show plot
plt.show()

In [ ]:
# RESULTS GROUPED WITH ERROR BANDS

# Set up plot
sns.set_theme(style=theme_style)
palette = sns.color_palette(color_palette_style)

plt.figure(figsize=(12, 6))

# Optimized function
sns.lineplot(
    data=df_optimized_all_days, x='duration_minutes', y='smoothed_median',
    color=palette[0], label='Optimized Function (Smoothed Median)', lw=2
)
plt.fill_between(
    df_optimized_all_days['duration_minutes'],
    df_optimized_all_days['smoothed_median'] - df_optimized_all_days['smoothed_std'],
    df_optimized_all_days['smoothed_median'] + df_optimized_all_days['smoothed_std'],
    color=palette[0], alpha=0.2, label='Optimized Std Dev'
)

# Baseline function
sns.lineplot(
    data=df_baseline_all_days, x='duration_minutes', y='smoothed_median',
    color=palette[1], label='Baseline Function (Smoothed Median)', lw=2
)
plt.fill_between(
    df_baseline_all_days['duration_minutes'],
    df_baseline_all_days['smoothed_median'] - df_baseline_all_days['smoothed_std'],
    df_baseline_all_days['smoothed_median'] + df_baseline_all_days['smoothed_std'],
    color=palette[1], alpha=0.2, label='Baseline Std Dev'
)

# Labels and title
plt.xlabel('Duration (min)')
plt.ylabel('Execution Time (ms)')
plt.title('Execution Time Comparison: Optimized vs. Baseline')

# Axis limits
plt.xlim(left=0)
plt.ylim(bottom=0)

# Legend
plt.legend()

# Layout adjustment
plt.tight_layout()

# SAVE PLOT
plot_filename = os.path.join(folder_to_save, "grouped-optimized-and-baseline-error-bands.pdf")
plt.savefig(plot_filename)

# Show plot
plt.show()

##### Calculate improvement

In [ ]:
# CALCULATE IMPROVEMENT OVER ALL DAYS


# Dictionary to store improvement values for each metric
metric_improvements = {}

# Loop over each date and read the corresponding file
for date in dates:
    data_dir = f"./logs_analysis/{date}/execution_{execution_count}/"
    file_path = os.path.join(data_dir, f"table.json")
    try:
        with open(file_path, "r") as file:
            data = json.load(file)
            for entry in data:
                metric = entry["Metric"]
                improvement = entry["% Improvement"]  # Keep original value
                
                if metric not in metric_improvements:
                    metric_improvements[metric] = []

                # Store the (date, improvement) tuple
                metric_improvements[metric].append((date, improvement))

    except FileNotFoundError:
        print(f"Warning: File for {date} not found, skipping.")

# Convert each metric's data into a Pandas DataFrame
tables = {}
for metric, values in metric_improvements.items():
    df = pd.DataFrame(values, columns=["Date", "% Improvement"])
    df.sort_values("Date", inplace=True)
    
    # Apply formatting (1 decimal place for % Improvement)
    styled_df = df.style.format({"% Improvement": "{:.2f}%"}) \
                        .set_table_styles([{
                            'selector': 'thead th', 
                            'props': [
                                ('font-size', '14px'), 
                                ('text-align', 'center'),
                                ('font-weight', 'bold')
                            ],
                        }]) \
                        .set_properties(**{
                            'text-align': 'center',
                            'font-size': '12px',
                            'padding': '7px',
                        }) \
                        .hide(axis="index")
    
    tables[metric] = styled_df  # Store formatted DataFrame for each metric

# Display each table in Jupyter Notebook
for metric, styled_df in tables.items():
    print(f"\n{metric}")
    display(styled_df)  # Shows the styled table nicely in Jupyter Notebook

# Compute and display average improvement for each metric
avg_improvements = {
    metric: sum(improvement for _, improvement in values) / len(values) 
    for metric, values in metric_improvements.items() if values
}

# Convert averages to a DataFrame and format
avg_df = pd.DataFrame(avg_improvements.items(), columns=["Metric", "Average % Improvement"])
styled_avg_df = avg_df.style.format({"Average % Improvement": "{:.2f}%"}) \
                            .set_table_styles([{
                                'selector': 'thead th', 
                                'props': [
                                    ('font-size', '14px'), 
                                    ('text-align', 'center'),
                                    ('font-weight', 'bold')
                                ],
                            }]) \
                            .set_properties(**{
                                'text-align': 'center',
                                'font-size': '12px',
                                'padding': '7px',
                            }) \
                            .hide(axis="index")

print("\nAverage Improvements Across All Dates")
display(styled_avg_df)

In [ ]:
# Convert metric_improvements dictionary into a DataFrame
df_list = []
for metric, values in metric_improvements.items():
    for date, improvement in values:
        df_list.append({"Date": date, "Metric": metric, "% Improvement": improvement})

df = pd.DataFrame(df_list)

# Convert Date column to datetime format for proper plotting
df["Date"] = pd.to_datetime(df["Date"])

# Define relevant metrics for plotting
relevant_metrics = ["Function Execution Time", "Download Duration", "Linear Regression Execution Time"]

# Filter data for the first plot
df_filtered = df[df["Metric"].isin(relevant_metrics)]

# Set Seaborn theme
theme_style = "whitegrid"
sns.set_theme(style=theme_style)

# Define color palette for consistency
num_days = df["Date"].nunique()
palette = sns.color_palette("Set2", n_colors=len(relevant_metrics))


# --- First Plot: All Three Metrics ---
plt.figure(figsize=(12, 6))

sns.lineplot(
    data=df_filtered, 
    x="Date", 
    y="% Improvement", 
    hue="Metric", 
    lw=2, 
    palette=palette,
    marker="o"
)

# Add labels and title
plt.xlabel("Date")
plt.ylabel("% Improvement")
plt.title("Improvement Over Time for Different Metrics")

# Format x-axis ticks
plt.xticks(rotation=45)

# Start y-axis at 0
# plt.ylim(bottom=0)

# Add a legend
plt.legend(title="Metric")

# Adjust layout
plt.tight_layout()

# Save Plot
plt.savefig("improvement_all_metrics.pdf")

# Show plot
plt.show()


# --- Second Plot: Function Execution Time vs. Linear Regression Execution Time ---
df_filtered2 = df[df["Metric"].isin(["Function Execution Time", "Linear Regression Execution Time"])]

plt.figure(figsize=(12, 6))

sns.lineplot(
    data=df_filtered2, 
    x="Date", 
    y="% Improvement", 
    hue="Metric", 
    lw=2, 
    palette=palette[:2],  # Use only the first two colors for consistency
    marker="o"
)

# Add labels and title
plt.xlabel("Date")
plt.ylabel("% Improvement")
plt.title("Comparison: Function Execution Time vs. Linear Regression Execution Time")

# Format x-axis ticks
plt.xticks(rotation=45)

# Start y-axis at 0
plt.ylim(bottom=0)

# Add a legend
plt.legend(title="Metric")

# Adjust layout
plt.tight_layout()

# Save Plot
plt.savefig("improvement_function_vs_linear_regression.pdf")

# Show plot
plt.show()

In [ ]:
# Set Seaborn theme
sns.set_theme(style="whitegrid")

# --- Boxplot for Each Metric ---
plt.figure(figsize=(12, 6))

sns.boxplot(
    data=df_filtered, 
    x="Metric", 
    y="% Improvement", 
    hue="Metric",  # Assign Metric to hue
    palette="Set2",  # Use the chosen palette
    dodge=False  # Avoid splitting the boxes by hue
)

# Add labels and title
plt.xlabel("Metric")
plt.ylabel("% Improvement")
plt.title("Distribution of % Improvement for Each Metric")

plt.ylim(bottom=0)


# Adjust layout
plt.tight_layout()

# Save Plot
plt.savefig("boxplot_improvement_by_metric.pdf")

# Show plot
plt.show()